<a href="https://colab.research.google.com/github/lucassaquett/VRPTHW/blob/main/VRPTHW_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.7 MB/s eta 0:00:00


In [17]:
from gurobipy import*
import numpy as np

In [63]:
#Nos e arcos
n = 11
clientes = [ i for i in range(n) if i !=0 ]
nos = [0]+clientes
arcos=[(i,j) for i in nos for j in nos if i!=j]

#Demanda
q={n:np.random.randint(10,15) for n in clientes}
q[0]=0

#Janelas de tempo
e={0:0, 1:10, 2:10, 3:10, 4:20, 5:20, 6:20, 7:40, 8:40, 9:40, 10:40} #Tempo minimo de chegada
l={0:200, 1:100, 2:100, 3:100, 4:150, 5:150, 6:150, 7:180, 8:180, 9:180, 10:180} #tempo maximo de chegada

s={n:np.random.randint(3,5) for n in clientes } #tempo de serviço
s[0] = 0

#Veiculos
veiculos = [1,2,3,4]
#q=50
Q={1:50,2:50,3:25,4:25}

#Coordenadas:
X=np.random.rand(len(nos))*100
Y=np.random.rand(len(nos))*100

custo = {(i,j) : np.hypot(X[i]-X[j],Y[i]-Y[j]) for i in nos for j in nos if i != j}
tempo = {(i,j) : np.hypot(X[i]-X[j],Y[i]-Y[j]) for i in nos for j in nos if i != j}

arco = [(i,j,k) for i in nos for j in nos for k in veiculos if i!=j ]
tempos_dos_arcos = [(i,k) for i in nos for k in veiculos]

In [47]:
model = Model('VRPTW')

In [57]:
#Variaveis de decições
x = model.addVars(arco, vtype=GRB.BINARY, name = 'x')
t = model.addVars(tempos_dos_arcos, vtype=GRB.CONTINUOUS, name = 't')

In [49]:
model.setObjective(quicksum(custo[i,j]*x[i,j,k] for i,j,k in arco), GRB.MINIMIZE)

In [50]:
#Restrições de chega e saída
model.addConstrs(quicksum(x[0,j,k] for j in clientes) <=1 for k in veiculos)
model.addConstrs(quicksum(x[i,0,k] for i in clientes) <=1 for k in veiculos)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [51]:
#Um veiuclo para cada nó
model.addConstrs(quicksum(x[i,j,k] for j in nos for k in veiculos if i!=j) == 1 for i in clientes)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>}

In [52]:
#Conservação de fluxo
model.addConstrs(quicksum(x[i,j,k] for j in nos if i!=j) - quicksum(x[i,j,k] for j in nos if i!=j) == 0 for i in nos for k in veiculos)

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 4): <gurobi.Constr *Awaiting Model Update*>,


In [53]:
#Capacidade do veiculo
model.addConstrs(quicksum(q[i]*quicksum(x[i,j,k] for j in nos if i!=j) for i in clientes)<=Q[k] for k in veiculos)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [59]:
#Restrições de tempo
model.addConstrs((x[i,j,k] == 1)>> (t[i,k]+s[i]+tempo[i,j] == t[j,k]) for i in clientes for j in clientes for k in veiculos if i!=j)
model.addConstrs(t[i,k] >= e[i] for i,k in tempos_dos_arcos)
model.addConstrs(t[i,k] <= l[i] for i,k in tempos_dos_arcos)

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 4): <gurobi.Constr *Awaiting Model Update*>,


In [60]:
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 242 rows, 1364 columns and 1056 nonzeros
Model fingerprint: 0x0ba05443
Model has 720 general constraints
Variable types: 484 continuous, 880 integer (880 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+01, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
  GenCon rhs range [1e+01, 9e+01]
  GenCon coe range [1e+00, 1e+00]

MIP start from previous solve produced solution with objective 179.638 (0.02s)
Loaded MIP start from previous solve with objective 179.638

Presolve removed 228 rows and 1324 columns
Presolve time: 0.01s
Presolved: 14 rows, 40 columns, 80 nonzeros
Variable types: 0 continuous, 40 integer (40 binary)

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)

In [61]:
print("Função Objetivo: ",str(round(model.ObjVal,2)))
for v in model.getVars():
  if v.x > 0.9:
    print(str(v.VarName)+"="+str(v.x))

Função Objetivo:  179.64
x[1,10,4]=1.0
x[2,0,4]=1.0
x[3,9,3]=1.0
x[4,1,3]=1.0
x[5,6,2]=1.0
x[6,5,2]=1.0
x[7,9,2]=1.0
x[8,3,2]=1.0
x[9,3,1]=1.0
x[10,1,1]=1.0
t[1,1]=10.0
t[1,2]=10.0
t[1,3]=10.0
t[1,4]=10.0
t[2,1]=10.0
t[2,2]=10.0
t[2,3]=10.0
t[2,4]=10.0
t[3,1]=10.0
t[3,2]=10.0
t[3,3]=10.0
t[3,4]=10.0
t[4,1]=20.0
t[4,2]=20.0
t[4,3]=20.0
t[4,4]=20.0
t[5,1]=20.0
t[5,2]=20.0
t[5,3]=20.0
t[5,4]=20.0
t[6,1]=20.0
t[6,2]=20.0
t[6,3]=20.0
t[6,4]=20.0
t[7,1]=40.0
t[7,2]=40.0
t[7,3]=40.0
t[7,4]=40.0
t[8,1]=40.0
t[8,2]=40.0
t[8,3]=40.0
t[8,4]=40.0
t[9,1]=40.0
t[9,2]=40.0
t[9,3]=40.0
t[9,4]=40.0
t[10,1]=40.0
t[10,2]=40.0
t[10,3]=40.0
t[10,4]=40.0
